In [1]:
import os
import platform
import pandas as pd
import tensorflow as tf

from modules.dataset_utils import DatasetUtils as dataset
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#########################################################################################################################################################
# PARAMETROS DEFINIDOS EN FUNCION DEL USUARIO
#########################################################################################################################################################

# Ruta hacia los CSV del dataset
train_file_relative = './data/700Train.csv'
validation_file_relative = './data/700Validation.csv'
test_file_relative = './data/700Test.csv'

# Ruta al directorio de imágenes con máscara aplicada
masked_dir_relative = '../local/output_masked/'

# Ruta relativa hacia la librería ai4eutils
ai4eutils_relative = "./repos/ai4eutils"

# Ruta relativa hacia la librería CameraTraps
CameraTraps_relative = "./repos/CameraTraps"

#########################################################################################################################################################
# PARAMETROS PARA CNN
#########################################################################################################################################################

IMG_HEIGHT = IMG_WEIGHT = 700
BATCH_SIZE = 64

#########################################################################################################################################################
#########################################################################################################################################################

home = os.path.expanduser("~")

ai4utils = os.path.abspath(ai4eutils_relative)
CameraTraps = os.path.abspath(CameraTraps_relative)
train_file = os.path.abspath(train_file_relative)
validation_file = os.path.abspath(validation_file_relative)
test_file = os.path.abspath(test_file_relative)
masked_dir = os.path.abspath(masked_dir_relative)

try:
    os.environ['PYTHONPATH']
except KeyError:
    os.environ['PYTHONPATH'] = ""
if platform.system() == 'Windows':
    os.environ['PYTHONPATH'] += (";" + ai4utils)
    os.environ['PYTHONPATH'] += (";" + CameraTraps)
else:
    os.environ['PYTHONPATH'] += (":" + ai4utils)
    os.environ['PYTHONPATH'] += (":" + CameraTraps)

print('=======================================================================================================================================')
print('PYTHONPATH: ' + os.environ['PYTHONPATH'])
print('')
print('ai4eutils PATH: ' + '\t' + ai4utils)
print('CameraTraps PATH: ' + '\t' + CameraTraps)
print('Train CSV PATH: ' + '\t'  + train_file)
print('Validation CSV PATH: ' + '\t' + validation_file)
print('Test CSV PATH: ' + '\t' + '\t' + test_file)
print('masked PATH: ' + '\t' + '\t' + masked_dir)
print('=======================================================================================================================================')


TensorFlow version: 2.9.2
Is GPU available? tf.test.is_gpu_available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

PYTHONPATH: /Users/carlos/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles:/Users/carlos/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles/lib/python:/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/ai4eutils:/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/CameraTraps

ai4eutils PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/ai4eutils
CameraTraps PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/repos/CameraTraps
Train CSV PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/data/700Train.csv
Validation CSV PATH: 	/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/data/700Validation.csv
Test CSV PATH: 		/Users/carlos/WORKSPACE/TFG-DeteccionFototrampeo/data/700Test.csv
masked PATH: 		/Users/carlos/WORKSPACE/local/output_masked


In [2]:
file_path, labels = dataset.load_dataset(train_file, masked_dir)
TRAIN_SAMPLES=len(labels)
train_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
train_dataset = train_dataset.map(dataset.load_image).map(dataset.resize_image).batch(BATCH_SIZE)

file_path, labels = dataset.load_dataset(validation_file, masked_dir)
VALIDATION_SAMPLES=len(labels)
validation_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
validation_dataset = validation_dataset.map(dataset.load_image).map(dataset.resize_image).batch(BATCH_SIZE)

file_path, labels = dataset.load_dataset(test_file, masked_dir)
TEST_SAMPLES=len(labels)
test_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
test_dataset = test_dataset.map(dataset.load_image).map(dataset.resize_image).batch(BATCH_SIZE)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-27 10:58:56.948736: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-27 10:58:56.948847: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
model = keras.Sequential([
    layers.Input((700, 700, 3)),
    layers.Conv2D(32, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [4]:
model.compile(
    loss = [keras.losses.BinaryCrossentropy(from_logits=False)],
    optimizer = keras.optimizers.Adam(),
    metrics =  ['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 698, 698, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 349, 349, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 347, 347, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 173, 173, 32)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 171, 171, 64)      18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 85, 85, 64)       0

In [5]:
model.fit(  train_dataset, 
            epochs=10, 
            steps_per_epoch=TRAIN_SAMPLES//BATCH_SIZE,
            validation_data=validation_dataset,
            validation_steps=VALIDATION_SAMPLES//BATCH_SIZE,
            batch_size=64
)

Epoch 1/10


2022-07-27 10:58:57.317957: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-27 10:58:57.636533: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/22 [>.............................] - ETA: 1:58 - loss: 1.6495 - accuracy: 0.6875

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x2cdcec400>
    label = <none> 
    device = <AGXG13GDevice: 0x1598ade00>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x179073e00>
        label = <none> 
        device = <AGXG13GDevice: 0x1598ade00>
            name = Apple M1 
    retainedReferences = 1


22/22 [==============================] - ETA: 0s - loss: 0.7311 - accuracy: 0.8629

2022-07-27 11:00:19.981210: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 88s 4s/step - loss: 0.7311 - accuracy: 0.8629 - val_loss: 0.6466 - val_accuracy: 0.9219
Epoch 2/10
22/22 [==============================] - 8s 284ms/step - loss: 0.3210 - accuracy: 0.9286 - val_loss: 0.5574 - val_accuracy: 0.9219


In [6]:
model.evaluate(test_dataset)

5/5 [==============================] - 7s 1s/step - loss: 0.4976 - accuracy: 0.9100


[0.49757447838783264, 0.909967839717865]